# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np
from ddpg import Agent
from buffer import ReplayBuffer
from collections import deque,namedtuple
import torch
import torch.nn.functional as F

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis_Windows_x86_64/Tennis.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode = False)[brain_name]
print(env_info.vector_observations)
# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.65278625 -1.5
  -0.          0.          6.83172083  6.         -0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.4669857  -1.5
   0.          0.         -6.83172083  6.          0.          0.        ]]
Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agents and receive feedback from the environment.

Once this cell is executed, you will watch the agents' performance, if they select actions at random with each time step.  A window should pop up that allows you to observe the agents.

Of course, as part of the project, you'll have to change the code so that the agents are able to use their experiences to gradually choose better actions when interacting with the environment!

In [5]:
for i in range(1, 6):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        print(actions)
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        print(next_states)
        rewards = env_info.rewards                         # get reward (for each agent)
        print(rewards)
        dones = env_info.local_done                        # see if episode finished
        print(dones)
        scores += env_info.rewards                         # update the score (for each agent)
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    print('Score (max over agents) from episode {}: {}'.format(i, np.max(scores)))                                        
    

[[ 0.49204278 -0.25776615]
 [-0.51804319  0.06901125]]
[[  0.           0.           0.           0.           0.
    0.           0.           0.          -7.38993645  -1.5
   -0.           0.           6.83172083   5.99607611  -0.
    0.          -5.91380835  -1.55886006  14.76128387  -0.98100001
    6.83172083   5.91759634  14.76128387  -0.98100001]
 [  0.           0.           0.           0.           0.
    0.           0.           0.          -6.70024681  -1.5
    0.           0.          -6.83172083   5.99607611   0.
    0.          -8.25437737  -1.55886006 -15.54129505  -0.98100001
   -6.83172083   5.91759634 -15.54129505  -0.98100001]]
[0.0, 0.0]
[False, False]
[[ 0.8105763  -1.        ]
 [ 0.47504541 -0.28777545]]
[[ -7.38993645  -1.5         -0.           0.           6.83172083
    5.99607611  -0.           0.          -5.91380835  -1.55886006
   14.76128387  -0.98100001   6.83172083   5.91759634  14.76128387
   -0.98100001  -3.48207974  -1.71581995  24.31728935  -1.9620

[[ -0.39999998  -0.11709452  30.           4.44940138   8.92170143
    1.97828746  30.           4.44940138  -3.267102     0.26898563
  -22.67102432   3.46840191   9.82170296   2.39349842 -22.67102432
    3.46840191  -1.48306227   0.55696583  17.84039879   2.48740244
   10.72170448   2.7106092   17.84039879   2.48740244]
 [ -8.16624546   0.24063723 -13.64923573  -3.39985418  -8.92170143
    1.97828746 -13.64923573  -3.39985418  -9.43944836  -0.15820813
  -12.73205376  -4.38085365  -9.82170296   2.39349842 -12.73205376
   -4.38085365  -8.90358353  -0.65515351   5.35862827  -5.36185312
  -10.72170448   2.7106092    5.35862827  -5.36185312]]
[0.0, 0.0]
[False, False]
[[-1.         -0.46496627]
 [-0.04160658  0.0042369 ]]
[[ -3.267102     0.26898563 -22.67102432   3.46840191   9.82170296
    2.39349842 -22.67102432   3.46840191  -1.48306227   0.55696583
   17.84039879   2.48740244  10.72170448   2.7106092   17.84039879
    2.48740244  -4.48306179   0.74684602 -30.           1.50640261
   1

[[-1.          0.05529748]
 [ 0.43633374  1.        ]]
[[ -6.35280466   0.26898634 -30.           3.46840191   6.69487906
    2.11524105 -30.           3.46840191  -5.43525791   0.55696654
    9.17546082   2.48740244   6.69487906   1.20638525   9.17546082
    2.48740244  -8.43525887   0.74684674 -30.           1.50640261
    5.89632797   1.72424316 -30.           1.50640261]
 [ -4.33373308   0.89535731  -3.33734465  -0.65179747  -6.69487906
    2.11524105  -3.33734465  -0.65179747  -7.3337326    0.7713176
  -30.          -1.63279748  -6.69487906   1.20638525 -30.
   -1.63279748  -6.07320976   0.27224034  13.09001255  -6.07551575
   -5.89632797   1.72424316  13.09001255  -6.07551575]]
[0.0, 0.0]
[False, False]
[[-0.74453165 -0.30914143]
 [ 1.         -0.38844096]]
[[ -5.43525791   0.55696654   9.17546082   2.48740244   6.69487906
    1.20638525   9.17546082   2.48740244  -8.43525887   0.74684674
  -30.           1.50640261   5.89632797   1.72424316 -30.
    1.50640261 -10.66885471   0.8

[[ -9.91398144  -1.13469064   9.85808372   6.41140032  -2.2036705
    5.23316336   9.85808372   6.41140032  -7.70583916  -0.55241048
   22.08140945   5.43040085  -3.10367084   5.13254356  22.08140945
    5.43040085 -10.6668644   -0.06823036 -29.61023712   4.44940138
   -4.00367117   4.93382359 -29.61023712   4.44940138]
 [ -2.33111215   0.26898658  -7.7308836    3.46840191   2.2036705
    5.23316336  -7.7308836    3.46840191  -0.39999998   0.55696678
   30.           2.48740244   3.10367084   5.13254356  30.
    2.48740244  -0.77635044   0.74684697  11.18247414   1.50640261
    4.00367117   4.93382359  11.18247414   1.50640261]]
[0.0, 0.0]
[False, False]
[[-1.         -0.47578362]
 [-1.          0.04404198]]
[[ -7.70583916  -0.55241048  22.08140945   5.43040085  -3.10367084
    5.13254356  22.08140945   5.43040085 -10.6668644   -0.06823036
  -29.61023712   4.44940138  -4.00367117   4.93382359 -29.61023712
    4.44940138 -10.90037727   0.31784981  -0.           3.46840191
   -4.90367031

[[-0.82108462  0.12572413]
 [-0.52529859  1.        ]]
[[ -3.601928    -1.85221565  12.0728035    0.          -9.46358013
    2.71405458  12.0728035    0.          -3.9985702   -1.852162
   -3.96641588   0.          -9.25422955   2.94711018  -3.96641588
    0.          -6.46182489  -1.852162   -24.63253784   0.
   -9.04487896   3.08206558 -24.63253784   0.        ]
 [-10.89980316  -1.85216224   0.           0.           9.46358013
    2.71405458   0.           0.          -9.77759075  -1.85216224
   11.22212887   0.           9.25422955   2.94711018  11.22212887
    0.         -10.9274931   -1.18355429   0.           6.41140032
    9.04487896   3.08206558   0.           6.41140032]]
[0.0, 0.0]
[False, False]
[[ 1.  1.]
 [-1.  1.]]
[[ -3.9985702   -1.852162    -3.96641588   0.          -9.25422955
    2.94711018  -3.96641588   0.          -6.46182489  -1.852162
  -24.63253784   0.          -9.04487896   3.08206558 -24.63253784
    0.          -3.46182561  -1.18355405  30.           6.41

...

[[ -3.19621205   0.7713176   17.06900597  -1.63279748   6.55063343
    1.0224613   17.06900597  -1.63279748  -0.85548633   0.54917783
   23.40725517  -2.61379719   6.66021919   0.74845207  23.40725517
   -2.61379719  -2.02899194   0.22893813 -10.28991985  -3.59479666
    7.208148     1.5895921  -10.28991985  -3.59479666]
 [ -3.34041357  -1.18355429   3.05528593   6.41140032  -6.55063343
    1.0224613    3.05528593   6.41140032  -6.3198657   -0.72232616
  -28.97263336  -0.62220097  -6.66021919   0.74845207 -28.97263336
   -0.62220097  -7.50043344  -0.84340632 -11.80568504  -1.60320103
   -7.208148     1.5895921  -11.80568504  -1.60320103]]
[0.0, 0.0]
[False, False]
[[ 1.         -1.        ]
 [-1.          0.50073463]]
[[ -0.85548633   0.54917783  23.40725517  -2.61379719   6.66021919
    0.74845207  23.40725517  -2.61379719  -2.02899194   0.22893813
  -10.28991985  -3.59479666   7.208148     1.5895921  -10.28991985
   -3.59479666  -0.39999998  -0.18940151  30.          -4.57579613
    

[[-3.61764216 -0.98316395 30.          6.21520042  7.76153517  5.85873604
  30.          6.21520042 -0.61764252 -0.42050385 30.          5.23420095
   7.76153517  5.6429162  30.          5.23420095 -0.8803224   0.0440563
   5.98387814  4.25320148  7.76153517  5.32899618  5.98387814  4.25320148]
 [-7.03280163 -0.98316395 -0.35365379  6.21520042 -7.76153517  5.85873604
  -0.35365379  6.21520042 -7.81229639 -0.42050385 -7.79495096  5.23420095
  -7.76153517  5.6429162  -7.79495096  5.23420095 -6.46722698  0.0440563
  13.45069408  4.25320148 -7.76153517  5.32899618 13.45069408  4.25320148]]
[0.0, 0.0]
[False, False]
[[-1. -1.]
 [-1. -1.]]
[[ -0.61764252  -0.42050385  30.           5.23420095   7.76153517
    5.6429162   30.           5.23420095  -0.8803224    0.0440563
    5.98387814   4.25320148   7.76153517   5.32899618   5.98387814
    4.25320148  -3.99999976   0.41051644 -30.           3.27220201
    7.76153517   4.91697645 -30.           3.27220201]
 [ -7.81229639  -0.42050385  -7.7949

[[  0.           0.           0.           0.           0.
    0.           0.           0.          -6.66271734  -1.5
   -0.           0.           6.04292488   5.94114017  -0.
    0.          -3.66271806  -1.55886006  30.          -0.98100001
    6.04292488   5.78418016  30.          -0.98100001]
 [  0.           0.           0.           0.           0.
    0.           0.           0.          -7.9240942   -1.5
    0.           0.          -6.04292488   5.94114017   0.
    0.         -10.92409611  -1.55886006 -30.          -0.98100001
   -6.04292488   5.78418016 -30.          -0.98100001]]
[0.0, 0.0]
[False, False]
[[ 0.22282344 -0.7215998 ]
 [ 0.36737261 -0.61226122]]
[[ -6.66271734  -1.5         -0.           0.           6.04292488
    5.94114017  -0.           0.          -3.66271806  -1.55886006
   30.          -0.98100001   6.04292488   5.78418016  30.
   -0.98100001  -2.9942472   -1.71581995   6.68470335  -1.96200001
    6.04292488   5.52912045   6.68470335  -1.96200001]
 [ 

[[-9.71081913e-01 -1.85216200e+00  3.00000000e+01 -4.76837158e-06
   3.45325184e+00  2.93825483e+00  3.00000000e+01 -4.76837158e-06
  -3.99999976e-01 -1.85216200e+00  3.00000000e+01 -4.76837158e-06
   2.55325150e+00  3.54395461e+00  3.00000000e+01 -4.76837158e-06
  -4.57256794e-01 -1.18355405e+00  2.71371593e+01  6.41140032e+00
   1.65325141e+00  4.05155468e+00  2.71371593e+01  6.41140032e+00]
 [-3.47377300e+00 -7.68052220e-01  2.47208166e+00 -7.18945074e+00
  -3.45325184e+00  2.93825483e+00  2.47208166e+00 -7.18945074e+00
  -4.73773360e-01 -1.54585707e+00  3.00000000e+01 -8.17045021e+00
  -2.55325150e+00  3.54395461e+00  3.00000000e+01 -8.17045021e+00
  -3.99999976e-01 -1.85242045e+00  3.00000000e+01  0.00000000e+00
  -1.65325141e+00  4.05155468e+00  3.00000000e+01  0.00000000e+00]]
[0.0, 0.0]
[False, False]
[[-1.          1.        ]
 [-0.33894578 -0.74987352]]
[[-3.99999976e-01 -1.85216200e+00  3.00000000e+01 -4.76837158e-06
   2.55325150e+00  3.54395461e+00  3.00000000e+01 -4.76837

[[ -5.7269578    0.22474802 -11.76846886  -3.39859676  -7.34674549
    3.73205662 -11.76846886  -3.39859676  -8.72695827  -0.17397165
  -30.          -4.37959623  -8.24674511   3.16055679 -30.
   -4.37959623 -10.94522381  -0.67079127  -0.          -5.3605957
   -9.14674664   2.49095726  -0.          -5.3605957 ]
 [ -8.45636559   0.72788727   5.97250986  -1.43659747   7.34674549
    3.73205662   5.97250986  -1.43659747  -7.25022936   0.5253675
   12.06135845  -2.41759729   8.24674511   3.16055679  12.06135845
   -2.41759729  -8.31863022   0.22474778 -10.68399525  -3.39859676
    9.14674664   2.49095726 -10.68399525  -3.39859676]]
[0.0, 0.0]
[False, False]
[[ 0.29866419 -1.        ]
 [ 1.          0.86168884]]
[[ -8.72695827  -0.17397165 -30.          -4.37959623  -8.24674511
    3.16055679 -30.          -4.37959623 -10.94522381  -0.67079127
   -0.          -5.3605957   -9.14674664   2.49095726  -0.
   -5.3605957  -10.04923248  -1.26571071   8.95992565  -6.34159517
  -10.04674816   1.723

When finished, you can close the environment.

In [6]:
def vector_avg(states):
    state_1 = np.array(states[0])
    state_2 = np.array(states[1])
    [state_1_1, state_1_2, state_1_3] = np.split(state_1, 3)
    [state_2_1, state_2_2, state_2_3] = np.split(state_2, 3)
    state_1 = (state_1_1 + state_1_2 + state_1_3)/3
    state_2 = (state_2_1 + state_2_2 + state_2_3)/3
    return [state_1, state_2]

class MADDPG():
    def __init__(self, discount_factor=0.95, tau=0.02):
        super(MADDPG, self).__init__()
        self.maddpg = [Agent(24, 256, 128, 2, 26, 512, 256),
                      Agent(24, 256, 128, 2, 26, 512, 256)]
        self.discount_factor = discount_factor
        self.tau = tau
        self.iter = 0
        self.memory = ReplayBuffer() #defined in the function setup
        self.t_step = 0
    def get_actors(self):
        local_actors = [agent.actor_local for agent in self.maddpg]
        return local_actors
    
    def get_target_actors(self):
        target_actors = [agent.actor_target for agent in self.maddpg]
        return target_actors
    
    def act(self, states, noise=0.0):
        actions = [agent.act(state) for agent, state in zip(self.maddpg, states)]
        return actions
    
    def target_act(self, states, noise=0.0):
        target_actions = [agent.target_act(state) for agent, state in zip(self.maddpg, states)]
        return target_actions
        
    def step(self, state, action, reward, next_state, done, Update_every = 1, Batch = 256):
        self.memory.add(state, action, reward, next_state, done)
        Update_times = 1
        self.t_step = (self.t_step + 1) % Update_every
        if len(self.memory) > Batch and self.t_step == 0:
            for _ in range(Update_times):
                experiences = self.memory.sample()
                [self.learn(experiences, agent, gamma = 0.99) for agent in range(2)]
    def learn(self, experiences, agent_number, gamma):
        states, actions, rewards, next_states, dones = experiences
        agent = self.maddpg[agent_number]
        agent.critic_optimizer.zero_grad()
        actions_next = torch.from_numpy(agent.target_act(next_states[agent_number])).detach().to(device)
        critic_input = torch.cat((next_states[agent_number], actions_next.float()), dim = 0).to(device)
        Q_targets_next = agent.critic_target(critic_input).detach()
        Q_targets = rewards + (gamma*Q_targets_next*(1-dones))
        critic_input = torch.cat((states[agent_number], actions[agent_number]), dim = 0).to(device)
        Q_expected = agent.critic_local(critic_input)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        agent.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(agent.critic_local.parameters(), 1)
        agent.critic_optimizer.step()
        actions_pred = (agent.act(states[agent_number].to('cpu').numpy()))
        actions_pred = torch.from_numpy(actions_pred).to(device)
        
        critic_input = torch.cat((states[agent_number].float(), (actions_pred).float()), dim = 0).to(device)
        actor_loss = -agent.critic_local(critic_input).mean()
        
        agent.actor_optimizer.zero_grad()
        actor_loss.backward()
        agent.actor_optimizer.step()
        
        self.soft_update(agent.critic_local, agent.critic_target)
        self.soft_update(agent.actor_local, agent.actor_target)
    def soft_update(self, local_model, target_model, tau=5e-3):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0 - tau)*target_param.data)
    


state = vector_avg(states)
print(state[0])
print(state[1])


[-2.04676962 -0.5         0.          0.         -2.37247276  1.98692004
  0.          0.        ]
[-2.65247377 -0.5         0.          0.          2.37247276  1.98692004
  0.          0.        ]


In [7]:
maddpg = MADDPG()

def main():
    print_every = 10
    scores_deque = deque(maxlen = 100)
    scores = []
    num_episodes = 5000
    episode_length = 80
    batchsize = 256
    noise = 2
    noise_reduction = 0.9999
    t_max = 300
    for episode in range(0,num_episodes):
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        [agent.reset() for agent in maddpg.maddpg]
        score = 0
        for t in range(t_max):
            actions = maddpg.act((states), noise = noise)
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards                         # get reward (for each agent)
            dones = env_info.local_done                        # see if episode finished
            score += max(env_info.rewards)
            maddpg.step(states, actions, rewards, next_states, dones)
            states = next_states
            if np.any(dones):                                  # exit loop if episode finished
                break
        scores_deque.append(score)
        scores.append(score)
        print("\r Episode {}\tAverage Score: {:.2f}".format(episode, np.mean(scores_deque)), end = "")
        torch.save(maddpg.maddpg[0].actor_local.state_dict(), "checkpoint_actor_agent1.pth")
        torch.save(maddpg.maddpg[0].critic_local.state_dict(), "checkpoint_critic_agent1.pth")
        torch.save(maddpg.maddpg[1].actor_local.state_dict(), "checkpoint_actor_agent2.pth")
        torch.save(maddpg.maddpg[1].critic_local.state_dict(), "checkpoint_critic_agent2.pth")
scores = main()

 Episode 17	Average Score: 0.00

C:\Users\Nikhil Masinete\Anaconda3\envs\drlnd\lib\site-packages\ipykernel_launcher.py:54: UserWarning: Using a target size (torch.Size([64, 2])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


 Episode 4999	Average Score: 0.00

In [8]:
print(actions)
print(states)
critic_input = np.concatenate((states[0], actions[0]), axis = None)
print(critic_input)
print(scores_deque)

[[-0.60007263 -0.67777037]
 [-0.31409705 -1.        ]]
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -6.14030886 -1.5
  -0.          0.         -7.11741829  5.96076012 -0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.         -7.9574213  -1.5
   0.          0.          7.11741829  5.96076012  0.          0.        ]]
[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.14030886 -1.5
 -0.          0.         -7.11741829  5.96076012 -0.          0.
 -0.60007263 -0.67777037]


NameError: name 'scores_deque' is not defined

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```